# RAG Multimodal

## Flickr 8k Dataset

In [2]:
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("adityajn105/flickr8k")

print("Path to dataset files:", path)

c:\Users\54Y1\Documents\SVII\recuperacionDeInformacion\RI_2\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.04G/1.04G [01:18<00:00, 14.1MB/s]

Extracting files...


Path to dataset files: C:\Users\54Y1\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1


In [4]:
path

'C:\\Users\\54Y1\\.cache\\kagglehub\\datasets\\adityajn105\\flickr8k\\versions\\1'

In [7]:
path = r"C:\Users\Asus\.cache\kagglehub\datasets\adityajn105\flickr8k\versions\1"

In [5]:
import os
import pandas as pd
from PIL import Image
from dotenv import load_dotenv

In [6]:
# Cargar variables de entorno
load_dotenv()

# --- 1. Cargar las descripciones ---
# La ruta 'path' que obtuviste de kagglehub apunta a la carpeta del dataset.
# Asegúrate de que la ruta sea correcta.
captions_path = os.path.join(path, "captions.txt")
df = pd.read_csv(captions_path)

# --- 2. Limpiar y estructurar los datos ---
# Agrupamos las descripciones por imagen.
captions_per_image = df.groupby('image')['caption'].apply(list).to_dict()

# Creamos una lista de rutas de imágenes y descripciones.
# Usaremos solo la primera descripción para simplificar.
image_dir = os.path.join(path, "Images")
image_paths = list(captions_per_image.keys())

# --- 3. Reducir el dataset para pruebas locales  ---
sample_size = 6000
image_paths_sample = image_paths[:sample_size]
captions_sample = {img: captions_per_image[img][0] for img in image_paths_sample}

print(f"Dataset listo. Usando una muestra de {len(image_paths_sample)} imágenes.")
# Ejemplo de una entrada
print(f"Ejemplo de imagen: {image_paths_sample[0]}")
print(f"Su descripción: {captions_sample[image_paths_sample[0]]}")

Dataset listo. Usando una muestra de 6000 imágenes.
Ejemplo de imagen: 1000268201_693b08cb0e.jpg
Su descripción: A child in a pink dress is climbing up a set of stairs in an entry way .


In [7]:
len(image_paths)

8091

In [8]:
import re

## CLIP y Pinecone

In [9]:
import torch
from transformers import CLIPProcessor, CLIPModel
from pinecone import Pinecone, ServerlessSpec

In [10]:
# --- 1. Cargar el modelo CLIP y moverlo a la GPU ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

model_id = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(model_id)
model = CLIPModel.from_pretrained(model_id, use_safetensors=True).to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Usando dispositivo: cuda


In [11]:
# --- 2. Inicializar Pinecone ---
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

# --- 3. Crear un índice vectorial en Pinecone [cite: 14] ---
# El índice almacenará los vectores.
index_name = "flickr8kragproject"
embedding_dim = model.config.projection_dim # Para este modelo CLIP es 512

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine", # La métrica de similitud estándar para CLIP
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )
    print(f"Índice '{index_name}' creado.")
else:
    print(f"Índice '{index_name}' ya existe.")

index = pc.Index(index_name)
print(index.describe_index_stats())

Índice 'flickr8kragproject' creado.
{'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


## Codificar e indexar el corpus

In [ ]:
!pip install cloudinary

In [22]:
import cloudinary
import cloudinary.uploader
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()
CLOUDINARY_API_KEY = os.environ.get("CLOUDINARY_API_KEY") 
CLOUDINARY_SECRET = os.environ.get("CLOUDINARY_SECRET")
CLOUD_NAME = "damiqc1pj"

# Configuración de Cloudinary
cloudinary.config(
  cloud_name = CLOUD_NAME,
  api_key = CLOUDINARY_API_KEY,
  api_secret = CLOUDINARY_SECRET,
  secure = True
)

In [14]:
from tqdm import tqdm # Para una barra de progreso

In [15]:
from PIL import Image

In [16]:
UPLOAD_LOG_FILE = "uploaded_log.txt"
# 1. Cargar la lista de imágenes ya subidas para no repetirlas
try:
    with open(UPLOAD_LOG_FILE, "r") as f:
        already_uploaded = set(line.strip() for line in f)
except FileNotFoundError:
    already_uploaded = set()

print(f"Se encontraron {len(already_uploaded)} imágenes ya subidas a Cloudinary.")

Se encontraron 0 imágenes ya subidas a Cloudinary.


In [17]:
batch_size = 32
vectors_to_upsert = []

In [23]:
# Abre el archivo de registro en modo 'append' para añadir nuevas entradas
with open(UPLOAD_LOG_FILE, "a") as log_file:
    for i in tqdm(range(0, len(image_paths_sample)), desc="Procesando imágenes"):
        filename = image_paths_sample[i]
        
        # 2. Revisa si la imagen ya fue subida antes de hacer nada
        if filename in already_uploaded:
            continue # Salta a la siguiente imagen

        # Si la imagen es nueva, se procesa
        local_path = os.path.join(image_dir, filename)
        
        try:
            # 3. Sube la imagen a Cloudinary
            upload_result = cloudinary.uploader.upload(local_path, public_id=os.path.splitext(filename)[0])
            image_url = upload_result['secure_url']
            
            # Procesa la descripción (si usas la función de preprocesamiento)
            caption = captions_sample[filename]
            # processed_caption = preprocesar_texto(caption) # Descomenta si lo usas

            # Codifica la imagen con CLIP
            image_pil = Image.open(local_path)
            with torch.no_grad():
                inputs = processor(images=[image_pil], return_tensors="pt").to(device)
                embedding = model.get_image_features(**inputs)
            
            embedding /= embedding.norm(dim=-1, keepdim=True)

            # Prepara el vector para Pinecone
            vectors_to_upsert.append({
                "id": filename,
                "values": embedding.cpu().numpy().tolist()[0],
                "metadata": {"caption": caption, "url": image_url}
            })
            
            # 4. Registra el éxito escribiendo el nombre del archivo en el log
            log_file.write(filename + "\n")
            already_uploaded.add(filename) # Añade al set en memoria también

        except Exception as e:
            print(f"Error al procesar {filename}: {e}")
            continue

        # Sube los vectores a Pinecone en lotes para mayor eficiencia
        if len(vectors_to_upsert) >= batch_size:
            index.upsert(vectors=vectors_to_upsert, namespace="images")
            vectors_to_upsert = [] # Limpia el lote

# Sube cualquier vector restante en el último lote
if vectors_to_upsert:
    index.upsert(vectors=vectors_to_upsert, namespace="images")

print("Proceso de indexación completado.")
print(f"Total de imágenes en Cloudinary (según registro): {len(already_uploaded)}")

Procesando imágenes:  63%|██████▎   | 3767/6000 [49:33<8:41:26, 14.01s/it] 

Error al procesar 3033257301_e2c8a39b04.jpg: Unexpected error - ProtocolError('Connection aborted.', ConnectionAbortedError(10053, 'Se ha anulado una conexión establecida por el software en su equipo host', None, 10053, None))
Error al procesar 3033612929_764d977bd5.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E265E1A90>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')


Procesando imágenes:  63%|██████▎   | 3773/6000 [49:33<2:26:14,  3.94s/it]

Error al procesar 3033668641_5905f73990.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E265E2490>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')
Error al procesar 3033686219_452b172ab0.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E265E2E90>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')
Error al procesar 3033741581_136889ac73.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused

Procesando imágenes:  63%|██████▎   | 3783/6000 [49:34<48:56,  1.32s/it]  

...

Procesando imágenes: 100%|█████████▉| 5986/6000 [1:08:40<00:00, 95.80it/s]

Error al procesar 3502897880_8392d0e4de.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E265E20D0>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')
Error al procesar 3502993968_4ee36afb0e.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E265E2AD0>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')
Error al procesar 3503011427_a4ee547c77.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused

Procesando imágenes: 100%|██████████| 6000/6000 [1:08:40<00:00,  1.46it/s]


Error al procesar 3506560025_8d0f4f9ac4.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E2753DE50>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')
Error al procesar 3506607642_40037b3fbf.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E2753E850>: Failed to resolve \'api.cloudinary.com\' ([Errno 11001] getaddrinfo failed)"))')
Error al procesar 3506869953_802f463178.jpg: Unexpected error - MaxRetryError('TCPKeepAliveHTTPSConnectionPool(host=\'api.cloudinary.com\', port=443): Max retries exceeded with url: /v1_1/damiqc1pj/image/upload (Caused

MaxRetryError: HTTPSConnectionPool(host='flickr8kragproject-y0ztyyn.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /vectors/upsert (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000024E4B0A1950>: Failed to resolve 'flickr8kragproject-y0ztyyn.svc.aped-4627-b74a.pinecone.io' ([Errno 11001] getaddrinfo failed)"))

In [21]:
batch_size = 32 # Reduce el batch size si tienes problemas de memoria o red
vectors_to_upsert = []

for i in tqdm(range(0, len(image_paths_sample), batch_size)):
    batch_limit = min(i + batch_size, len(image_paths_sample))
    batch_filenames = image_paths_sample[i:batch_limit]
    
    # Prepara el lote de imágenes y metadatos
    batch_images = []
    batch_metadata = []
    
    for filename in batch_filenames:
        local_path = os.path.join(image_dir, filename)
        
        # 1. Subir imagen a Cloudinary
        try:
            upload_result = cloudinary.uploader.upload(local_path, public_id=os.path.splitext(filename)[0])
            image_url = upload_result['secure_url']
            
            # Añadir imagen y metadatos al lote solo si la subida es exitosa
            batch_images.append(Image.open(local_path))
            batch_metadata.append({
                "caption": captions_sample[filename],
                "url": image_url # <-- Almacenamos la URL de Cloudinary
            })
            
        except Exception as e:
            print(f"Error al subir {filename}: {e}")
            continue

    # Si el lote no está vacío después de las subidas
    if batch_images:
        # 2. Codificar las imágenes con CLIP
        with torch.no_grad():
            inputs = processor(images=batch_images, return_tensors="pt", padding=True).to(device)
            image_embeddings = model.get_image_features(**inputs)

        image_embeddings /= image_embeddings.norm(dim=-1, keepdim=True)

        # 3. Preparar los datos para Pinecone
        for j, metadata in enumerate(batch_metadata):
            embedding = image_embeddings[j].cpu().numpy().tolist()
            # Usamos el nombre del archivo original como ID
            file_id = os.path.basename(metadata['url']).split('.')[0] + '.jpg'
            
            vectors_to_upsert.append({
                "id": file_id,
                "values": embedding,
                "metadata": metadata
            })

# 4. Subir (Upsert) los datos restantes a Pinecone
if vectors_to_upsert:
    # Subimos en lotes más pequeños para no sobrecargar la API
    for k in range(0, len(vectors_to_upsert), 100):
        index.upsert(vectors=vectors_to_upsert[k:k+100], namespace="images")
    print(f"Indexación finalizada. Total de {len(vectors_to_upsert)} vectores subidos.")

print(index.describe_index_stats())

 47%|████▋     | 89/188 [00:00<00:00, 451.49it/s]

Error al subir 1000268201_693b08cb0e.jpg: Must supply api_secret
Error al subir 1001773457_577c3a7d70.jpg: Must supply api_secret
Error al subir 1002674143_1b742ab4b8.jpg: Must supply api_secret
Error al subir 1003163366_44323f5815.jpg: Must supply api_secret
Error al subir 1007129816_e794419615.jpg: Must supply api_secret
Error al subir 1007320043_627395c3d8.jpg: Must supply api_secret
Error al subir 1009434119_febe49276a.jpg: Must supply api_secret
Error al subir 1012212859_01547e3f17.jpg: Must supply api_secret
Error al subir 1015118661_980735411b.jpg: Must supply api_secret
Error al subir 1015584366_dfcec3c85a.jpg: Must supply api_secret
Error al subir 101654506_8eb26cfb60.jpg: Must supply api_secret
Error al subir 101669240_b2d3e7f17b.jpg: Must supply api_secret
Error al subir 1016887272_03199f49c4.jpg: Must supply api_secret
Error al subir 1019077836_6fc9b15408.jpg: Must supply api_secret
Error al subir 1019604187_d087bf9a5f.jpg: Must supply api_secret
Error al subir 1020651753_0

100%|██████████| 188/188 [00:00<00:00, 451.30it/s]

Error al subir 2752809449_632cd991b3.jpg: Must supply api_secret
Error al subir 2752919987_8bfca604ab.jpg: Must supply api_secret
Error al subir 2752923489_d3bceebacb.jpg: Must supply api_secret
Error al subir 2752926645_801a198ff6.jpg: Must supply api_secret
Error al subir 2753506871_dc38e7d153.jpg: Must supply api_secret
Error al subir 2753531542_ace2c870b7.jpg: Must supply api_secret
Error al subir 275401000_8829250eb3.jpg: Must supply api_secret
Error al subir 2754271176_4a2cda8c15.jpg: Must supply api_secret
Error al subir 2754898893_95239c1f19.jpg: Must supply api_secret
Error al subir 2755053974_5cc157512e.jpg: Must supply api_secret
Error al subir 275516348_cbccebc125.jpg: Must supply api_secret
Error al subir 2755314937_1e974bf2b5.jpg: Must supply api_secret
Error al subir 2755362721_368cbde668.jpg: Must supply api_secret
Error al subir 2755952680_68a0a1fa42.jpg: Must supply api_secret
Error al subir 2756591658_3ca6db1595.jpg: Must supply api_secret
Error al subir 2756636539_c

{'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


## Búsqueda Multimodal

In [19]:
from IPython.display import display, Image as IPImage

In [52]:
def search(query, top_k=5):
    """
    Realiza una búsqueda multimodal (texto o imagen) en el índice de Pinecone.
    """
    if isinstance(query, str): # Si la consulta es texto
        # Codificar el texto de la consulta
        with torch.no_grad():
            inputs = processor(text=[query], return_tensors="pt").to(device)
            query_embedding = model.get_text_features(**inputs)
    
    elif isinstance(query, Image.Image): # Si la consulta es una imagen
        # Codificar la imagen de la consulta
        with torch.no_grad():
            inputs = processor(images=[query], return_tensors="pt").to(device)
            query_embedding = model.get_image_features(**inputs)
    else:
        raise ValueError("La consulta debe ser un string de texto o un objeto PIL.Image")

    # Normalizar el vector de consulta y convertir a lista
    query_embedding /= query_embedding.norm(dim=-1, keepdim=True)
    query_vector = query_embedding.cpu().numpy().tolist()[0]
    
    # --- Realizar la búsqueda en Pinecone ---
    results = index.query(
        vector=query_vector,
        top_k=top_k,
        include_metadata=True,
        namespace="images"
    )
    
    # --- Mostrar los resultados recuperados [cite: 18] ---
    retrieved_items = []
    print(f"Consulta: '{query if isinstance(query, str) else 'Imagen de entrada'}'")
    for match in results['matches']:
        metadata = match['metadata']
        print(f"  - Score: {match['score']:.4f}")
        print(f"    Descripción: {metadata['caption']}")
        print(f"    URL: {metadata['url']}")
        
        # Mostrar la imagen directamente desde la URL en un notebook
        display(IPImage(url=metadata['url'], width=300))
        
        retrieved_items.append(metadata)
        
    return retrieved_items

In [86]:
# --- Ejemplo de uso ---
# Búsqueda por texto
text_results = search("gangster", top_k=3)

Consulta: 'gangster'
  - Score: 0.2531
    Descripción: A man in a blue shirt , cowboy hat and a cross neckless walking in front of a carnival or amusement park ride .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753935813/1396703063_e8c3687afe.jpg


  - Score: 0.2464
    Descripción: A girl plays on a red toy .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753936046/1794818900_e0ffdd268e.jpg


  - Score: 0.2432
    Descripción: a attrative woman climbs over a fence .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753936135/2039457436_fc30f5e1ce.jpg


In [60]:
# Búsqueda por imagen (carga una imagen de tu dataset para probar)
try:
    image_query_path = os.path.join(image_dir, image_paths_sample[50])
    image_query = Image.open(image_query_path)
    image_results = search(image_query, top_k=3)
except (FileNotFoundError, IndexError):
    print("No se pudo cargar la imagen de ejemplo para la búsqueda.")

Consulta: 'Imagen de entrada'
  - Score: 0.9998
    Descripción: A black and white dog is attempting to catch a yellow and purple object in a low cut yard .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753935281/1067180831_a59dc64344.jpg


  - Score: 0.8603
    Descripción: A black and white dog catches a toy in midair .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753935581/1072153132_53d2bb1b60.jpg


  - Score: 0.8174
    Descripción: a black and white dog jumping in the air to get a toy .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753935261/1020651753_06077ec457.jpg


In [99]:
from PIL import Image
import os

# Ruta absoluta o relativa de tu imagen local
image_query_path = "C:/Users/Asus/Downloads/childs.jpg"

try:
    image_query = Image.open(image_query_path).convert("RGB")
    image_results = search(image_query, top_k=3)
except FileNotFoundError:
    print(f"No se encontró la imagen: {image_query_path}")
except Exception as e:
    print(f"Error al abrir o procesar la imagen: {e}")


Consulta: 'Imagen de entrada'
  - Score: 0.7908
    Descripción: A little girl with arms outstretched plays with others at a sprinkler park .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753935699/1259936608_e3f0064f23.jpg


  - Score: 0.7575
    Descripción: A little boy jumps on a toy air gun .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753940506/2100735137_05c6079537.jpg


  - Score: 0.7512
    Descripción: A boy wearing a red shirt dumps water out of a yellow bucket onto a girl 's head .
    URL: https://res.cloudinary.com/dblwkwcyf/image/upload/v1753940589/2172526745_649f420569.jpg


## RAG

In [49]:
from openai import OpenAI
from IPython.display import Markdown, display, Image as IPImage

In [ ]:
client_deepseek = OpenAI(api_key="", base_url="")

In [37]:
# Asegúrate de haber inicializado tu cliente de DeepSeek previamente
# from deepseek import DeepseekClient
# client_deepseek = DeepseekClient(api_key="TU_DEEPSEEK_API_KEY")

def generate_narrative_deepseek(query, retrieved_data):
    """
    Genera una respuesta narrativa usando DeepSeek y el contexto recuperado.
    """
    # 1. Concatenar las descripciones recuperadas para formar un contexto.
    #    Esto cumple con el requisito 'c)' del proyecto[cite: 20].
    contexto = "\n".join([f"- {item['caption']}" for item in retrieved_data])

    # 2. Crear el prompt adaptado al objetivo de generar una narrativa.
    prompt_narrativo = (
        f"Considerando la consulta de entrada '{query}', usa las siguientes descripciones de imágenes "
        f"recuperadas como contexto para escribir una breve historia o una descripción explicativa y coherente. "
        f"Sintetiza la información en un párrafo fluido.\n\n"
        f"Contexto:\n{contexto}"
    )

    print("\n--- Prompt enviado a DeepSeek ---")
    print(prompt_narrativo)

    # 3. Realizar la llamada a la API de DeepSeek.
    response_deepseek = client_deepseek.chat.completions.create(
        model="deepseek/deepseek-r1:free", # O el modelo que estés usando
        messages=[
            {"role": "system", "content": "Eres un asistente experto en generar narrativas creativas y descriptivas basadas en un contexto de imágenes."},
            {"role": "user", "content": prompt_narrativo}
        ],
        temperature=0.7 # Una temperatura ligeramente más alta puede favorecer la creatividad
    )
    
    # 4. Extraer y devolver la respuesta generada.
    return response_deepseek.choices[0].message.content

In [38]:
# --- Ejemplo de uso del pipeline completo ---
# Asumimos que la función search() del paso anterior ya existe
query_texto = "a dog running on a beach"
retrieved_context = search(query_texto, top_k=3)

if retrieved_context:
    final_response = generate_narrative_deepseek(query_texto, retrieved_context)
    
    print("\n--- Respuesta Final Generada por DeepSeek ---")
    # Usamos display(Markdown()) si estás en un notebook para una mejor visualización
    # from IPython.display import display, Markdown
    # display(Markdown(final_response))
    print(final_response)

Consulta: 'a dog running on a beach'
  - ID: 1799271536_6e69c8f1dc.jpg (Score: 0.3397)
    Descripción: A brown and black dog runs down a sandy beach .
  - ID: 140430106_2978fda105.jpg (Score: 0.3329)
    Descripción: A dog is running along a beach on a sunny day .
  - ID: 1057251835_6ded4ada9c.jpg (Score: 0.3313)
    Descripción: A light-colored dog runs on the beach .

--- Prompt enviado a DeepSeek ---
Considerando la consulta de entrada 'a dog running on a beach', usa las siguientes descripciones de imágenes recuperadas como contexto para escribir una breve historia o una descripción explicativa y coherente. Sintetiza la información en un párrafo fluido.

Contexto:
- A brown and black dog runs down a sandy beach .
- A dog is running along a beach on a sunny day .
- A light-colored dog runs on the beach .

--- Respuesta Final Generada por DeepSeek ---
Bajo el cálido resplandor del sol, un perro de pelaje multicolor, con matices que van del marrón terroso al negro azabache mezclados c

## Prueba con cloudinary

In [43]:
load_dotenv()
CLOUDINARY_API_KEY = os.environ.get("CLOUDINARY_API_KEY") 
CLOUDINARY_SECRET = os.environ.get("CLOUDINARY_SECRET")
CLOUD_NAME = os.environ.get("CLOUDINARY_NAME")

In [ ]:
import cloudinary
import cloudinary.uploader
from cloudinary.utils import cloudinary_url

# Configuration       
cloudinary.config( 
    cloud_name = CLOUD_NAME, 
    api_key =   CLOUDINARY_API_KEY,
    api_secret = CLOUDINARY_SECRET, # Click 'View API Keys' above to copy your API secret
    secure=True
)

# Upload an image
upload_result = cloudinary.uploader.upload("https://res.cloudinary.com/demo/image/upload/getting-started/shoes.jpg",
                                           public_id="shoes")
print(upload_result["secure_url"])

# Optimize delivery by resizing and applying auto-format and auto-quality
optimize_url, _ = cloudinary_url("shoes", fetch_format="auto", quality="auto")
print(optimize_url)

# Transform the image: auto-crop to square aspect_ratio
auto_crop_url, _ = cloudinary_url("shoes", width=500, height=500, crop="auto", gravity="auto")
print(auto_crop_url)

  Using cached cloudinary-1.44.1-py3-none-any.whl.metadata (8.0 kB)
Using cached cloudinary-1.44.1-py3-none-any.whl (147 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
from IPython.display import Image, display

url = "https://res.cloudinary.com/demo/image/upload/sample.jpg"
display(Image(url=url))